# Chapter 2: Programming One-Dimensional Finite Element Method for the Linear Boundary Value Problem

## Listing 2.11: Python code implementing libraries import

In [26]:
import numpy
import scipy.sparse as sparse
import timeit
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

## Listing 2.12: Python code for Gaussian quadrature rule

In [ ]:
def GaussQuad(n,s0,s1):
    nargin=len(locals())
    d=[]; w=[]
    if nargin==1:
        s0=-1; s1=1
    if n==1:
        d=(s0+s1)/2; w=s1-s0  
        return d,w 
    j=numpy.arange(1,n)
    bta=j/numpy.sqrt(4*(j**2)-1)   
    Q=numpy.diag(bta, k=-1)+numpy.diag(bta, k=1)
    L,U = numpy.linalg.eig(Q)
    k=numpy.argsort(L)
    d=L[k]; w=2*(U[0][k]**2)
    d=s0+0.5*(s1-s0)*(d+1)
    w=0.5*(s1-s0)*w
    return d,w

## Listing 2.13: Python code for Lobatto quadrature rule

In [ ]:
def LobattoQuad (n,s0,s1):
    nargin=len(locals())
    d=[]; w=[]
    if n<2:
        print('LobattoQuad: number of nodes is less then 2') 
        return d,w 
    if nargin==1:
        s0=-1; s1=1 
    if n==2:
        d=[s0,s1]; w=[(s1-s0)/2,(s1-s0)/2]  
        return d,w 
    k=numpy.arange(1,n-2)
    bta=numpy.sqrt(k*(k+2)/(2*k+1)/(2*k+3)) 
    Q=numpy.diag(bta,k=-1)+numpy.diag(bta,k=1) 
    L,U = numpy.linalg.eig(Q)
    k=numpy.argsort(L)
    d=L[k]; w=4/3*(U[0][k]**2); w=w/(1-d**2)
    w=numpy.append(w,2/(n**2-n))
    w=numpy.insert(w,0,2/(n**2-n))
    d=numpy.append(d,1)
    d=numpy.insert(d,0,-1)
    d=s0+0.5*(s1-s0)*(d+1);
    w=0.5*(s1-s0)*w;
    return d,w

## Listing 2.14: Python code for interpolating and differentiating matrices calculation

In [ ]:
def interpolatingMat(x,d):
    nx=numpy.size(x); nd=numpy.size(d)
    E=numpy.zeros((nd,nx)) 
    D=numpy.zeros((nd,nx))
    b=numpy.zeros((nx,))
    j_all=numpy.arange(nx)
    for i in range(nx):
        j=numpy.delete(j_all,i) 
        b[i]=1/numpy.prod(x[i]-x[j]) 
    for i in range(nx):
        j=numpy.delete(j_all,i)
        for k in range(nd):
            E[k,i]=b[i]*numpy.prod(d[k]-x[j])
            ds=0
            for l in j:
                i1=min(i,l); i2=max(i,l) 
                jj=numpy.delete(j_all,[i1,i2]) 
                ds=ds+numpy.prod(d[k]-x[jj]) 
            D[k,i]=b[i]*ds
    return E,D

## Listing 2.15: Python code for the FEM error analysis

In [ ]:
def errorNorm(u,du,y,xl,d):
    m=numpy.size(d)-1
    d_new,w=GaussQuad(m+2,0,1)
    E,D=interpolatingMat(d,d_new) 
    e0=0; e1=0
    for l in range(numpy.size(xl)-1):
        h=xl[l+1]-xl[l]
        dl=xl[l]+h*d_new
        ic=l*m+numpy.arange(m+1)
        uh=E @ y[ic]; duh=D @ y[ic]/h 
        e=u(dl)-uh; de=du(dl)-duh
        e0=e0+h/2*sum(w*(e**2))
        e1=e1+h/2*sum(w*(de**2))
    e0=e0**(1/2); e1=e1**(1/2)
    return e0,e1


## Listing 2.16: Python code for generating values on refined mesh

In [ ]:
def getRefinedValues(u,xl,d,k):
    d_ref=numpy.linspace(0,1,k+1) # refined basis element
    E,D=interpolatingMat(d,d_ref)
    n=numpy.size(xl); m=numpy.size(d)-1
    x_ref=(n-1)*k+1; # number of all refined mesh points
    x=numpy.zeros((x_ref,))
    y=numpy.zeros(x.shape)
    dy=numpy.zeros(x.shape)
    for l in range(n-1):
        h=xl[l+1]-xl[l]  #size of element l
        i_c=l*m+numpy.arange(m+1) # coarse point indices
        i_ref=l*k+numpy.arange(k+1)  # refined points indices
        x[i_ref]=numpy.linspace(xl[l],xl[l+1],k+1)
        y[i_ref]=E @ u[i_c] 
        dy[i_ref]=D @ u[i_c]/h 
    return x,y,dy


## Listing 2.17: Python code for assembling one-dimensional matrices

In [ ]:
def assemblingAF(c,b,a,f,xl,x,d,w):
    E,D=interpolatingMat(x,d)
    ET=numpy.transpose(E)
    DT=numpy.transpose(D)
    m=numpy.size(x)-1
    n=numpy.size(xl)-1
    n_total=n*m+1;
    A=sparse.coo_matrix((n_total,n_total))
    F=sparse.coo_matrix((n_total,1))
    i_loc=numpy.arange(m + 1)
    i_loc_sp_mat = numpy.repeat(i_loc, m + 1)
    j_loc_sp_mat = numpy.tile(i_loc, m + 1)
    for l in range(n):
        h=xl[l+1]-xl[l] # size of element l
        xd=xl[l]+h*d # quadrature nodes on element l
        c_loc=numpy.diag((w/h)*c(xd))
        b_loc=numpy.diag(w*b(xd))
        a_loc=numpy.diag((h*w)*a(xd))
        Al=(DT @ c_loc+ET @ b_loc) @ D+ET @ a_loc @ E
        row=l*m+i_loc_sp_mat
        col=l*m+j_loc_sp_mat
        A=A+sparse.coo_matrix((Al.flatten(),(row, col)),\
            shape=(n_total,n_total))
        Fl=ET @ (h*w*f(xd)).reshape(-1,1)
        F=F+sparse.coo_matrix((Fl.flatten(),\
            (l*m+i_loc, numpy.zeros(m+1))),shape=(n_total,1))
    return A,F

## Listing 2.18: Python code for assembling one-dimensional boundary conditions

In [ ]:
def assemblingBC(bc,A,F):
    # bc - boundary conditions matrix of size 2x3 :
    # bc(1,:)=[1,0,u_s0] , or bc(1,:)=[3,sig_s0,mu_s0]
    # bc(2,:)=[1,0,u_s1] , or bc(2,:)=[3,sig_s1,mu_s1]
    n=F.shape[0]-1 #number of all mesh points
    # set boundary conditions of 3-d type
    if bc[0,0]==3:
        A[0,0]=A[0,0]+bc[0,1]
        F[0]=F[0]+bc[0,2]
    if bc[1,0]==3:
        A[n,n]=A[n,n]+bc[1,1] 
        F[n,0]=F[n,0]+bc[1,2]
    #set boundary conditions of the first type
    ib=0; ie=n+3
    if bc[0,0]==1: 
        ib=1; u_s0=bc[0,2] 
        F=F-u_s0*A[:,0]
    if bc[1,0]==1: 
        ie=n+2; u_s1=bc[1,2] 
        F=F-u_s1*A[:,n]
    A0=A[ib:ie,ib:ie] 
    F0=F[ib:ie,0]
    return A0,F0

## Listing 2.19: Python code for solving one-dimensional FEM system

In [ ]:
def solveFEM(bc,A0,F0):
    u_s0 =[]; u_s1 =[] 
    if bc[0,0]==1: u_s0=bc[0,2] 
    if bc[1,0]==1: u_s1=bc[1,2]
    u0=sparse.linalg.spsolve(A0,F0)
    u=u_s0
    u=numpy.append(u,u0)
    u=numpy.append(u,u_s1)
    return u

## Listing 2.20: Python code for solving one-dimensional BVP

In [ ]:
def solveBVP(bc,c,b,a,f,xl,x,d,w):
    m=numpy.size(x)-1
    n=numpy.size(xl) 
    N=(n-1)*m+1 # number of mesh points
    x_mesh=numpy.zeros((N,)) # all mesh points
    for l in range(n-1):
        ne=l*m+numpy.arange(m+1) # mesh point indices
        x_mesh[ne]=xl[l]+(xl[l+1]-xl[l])*x 
    A,F=assemblingAF(c,b,a,f,xl,x,d,w)
    A0,F0=assemblingBC(bc,A,F)
    u=solveFEM(bc,A0,F0)
    return u,x_mesh

## Listing 2.21: Python code for solving example one-dimensional BVP

In [ ]:
def mainTest():
    # set BVP data:
    # integration domain
    s_0=-1; s_1=1 
    # exact solution
    def u(x): return numpy.sin(x) 
    def du(x): return numpy.cos(x) 
    # equation coefficients
    def c(x): return numpy.cos(x)
    def b(x): return -2*numpy.sin(x) 
    def a(x): return numpy.sin(x)
    def f(x): return numpy.sin(x)**2
    # boundary conditions
    sigma=0
    mu=c(s_1)*du(s_1)+sigma*u(s_1)
    bc=numpy.array([[1, 0, u(s_0)],
                     [3, sigma, mu] ])    
    # set FEM parameters
    m=3 # polynomial degree
    ne=6 # number of final elements
    M=m+1 # number of quadrature nodes
    xl=numpy.linspace(s_0,s_1,ne+1) # finite element nodes
    t,_ =LobattoQuad(m+1,0,1) # interpolation nodes
    d,w=LobattoQuad(M,0,1) # quadrature nodes and weighs
    # solve BVP
    start_time = timeit.default_timer()
    uh,x=solveBVP(bc,c,b,a,f,xl,t,d,w) 
    print("solving time =",timeit.default_timer() - start_time)
    # find error in L_2 and H^1 norms
    e0,e1=errorNorm(u,du,uh,xl,t)
    print("L2 norm error =",e0)
    print("H1 norm error =",e1)
    X,Uh,dUh=getRefinedValues(uh,xl,t,10*m) 
    # find error in various norms
    Z=u(X)-Uh; z=u(x)-uh; dZ=du(X)-dUh 
    ie=numpy.arange(0,numpy.size(x),m) 
    print("uh error inf norm:",numpy.linalg.norm(Z,numpy.inf))
    print("duh error inf norm:",numpy.linalg.norm(dZ,numpy.inf))
    print("error at f.e. nodes inf norm:", \
        numpy.linalg.norm(u(xl)-uh[ie],numpy.inf))
    plt.figure() 
    # plot solution
    plt.plot(X,Uh,'-b',x,uh,'rx',x[ie],uh[ie],'ro')
    plt.xlabel('x'); plt.ylabel('u_h')
    plt.show()
    #plot solution error
    plt.plot(X,Z,'-b',x,z,'rx',x[ie],z[ie],'ro')
    plt.xlabel('x'); plt.ylabel('u-u_h')
    plt.show()
    #plot derivatives error
    plt.plot(X,dZ,'-b',\
            x,numpy.zeros(numpy.size(x)),'rx',\
            x[ie],numpy.zeros(numpy.size(x[ie])),'ro')
    plt.xlabel('x'); plt.ylabel('u''-u''_h'); 
    plt.show()

## Run `mainTest` function:

In [ ]:
mainTest()